In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
from numpy import genfromtxt
import itertools,random
# np.set_printoptions(threshold=np.nan)
# np.set_printoptions(threshol

Read Embedding File

In [2]:
df=pd.read_csv('../node2vec/emb/sampleandhold_10.emb',skiprows=1,sep=' ',header=None)
df=df.sort_values(by=[0],ascending=True)
df=df.rename(columns = {0:'node'})

Read Graph

In [3]:
G=nx.read_edgelist('../datasets/fb_test.txt.10_sampleandhold',nodetype=int)
edgeLen=len(G.edges())
edges_G=G.edges()
edges_G=np.asarray(edges_G,dtype=int)
np.random.shuffle(edges_G)

Create Egde Feature vectors for positive class

In [4]:
edge_features_1=np.empty(shape=[0,len(df.columns)-1])
edgeNO=0
for edge in edges_G:
    edgeNO+=1
    node1=df[df['node']==edge[0]].as_matrix().astype(np.float32)[0][1:]
    node2=df[df['node']==edge[1]].as_matrix().astype(np.float32)[0][1:]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_1=np.vstack([edge_features_1,hadamard])

In [5]:
G_neg=nx.Graph()
sample_len=len(G.edges())
E=G.edges()
while(len(G_neg.edges())<sample_len):
    n1=random.choice(G.nodes())
    n2=random.choice(G.nodes())
    if(G.has_edge(n1,n2)==False):
        G_neg.add_edge(n1,n2)

In [6]:
edge_features_0=np.empty(shape=[0,len(df.columns)-1])

for edge in G_neg.edges():
    node1=df[df['node']==edge[0]].as_matrix().astype(np.float32)[0][1:]
    node2=df[df['node']==edge[1]].as_matrix().astype(np.float32)[0][1:]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_0=np.vstack([edge_features_0,hadamard])

In [7]:
class_0=np.zeros([edgeLen,1])
class_1=np.ones([edgeLen,1])
edge_features_0=np.hstack([edge_features_0,class_0])
edge_features_1=np.hstack([edge_features_1,class_1])
dataset=np.vstack([edge_features_0,edge_features_1])

In [11]:
np.random.shuffle(dataset)
train_X=dataset[:,:dataset.shape[1]-2]
train_Y=dataset[:,dataset.shape[1]-1:]

In [16]:
G_test=nx.read_adjlist('../datasets/fb_train.txt',nodetype=int)
test_0=np.empty(shape=[0,len(df.columns)-1])
test_1=np.empty(shape=[0,len(df.columns)-1])
print len(test_0)
sample_len=len(df)
i=0
while(len(test_0)<sample_len and len(test_1)<sample_len):
    i+=1
    n1=random.choice(G_test.nodes())
    n2=random.choice(G_test.nodes())
    if(len(df[df['node']==n1])==0 or len(df[df['node']==n2])==0):
        pass
    else:
        node1=df[df['node']==n1].as_matrix().astype(np.float32)[0][1:]
        node2=df[df['node']==n2].as_matrix().astype(np.float32)[0][1:]
        avg=np.mean([node1,node2],axis=0)
        hadamard=np.multiply(node1,node2)
        diff=node1-node2
        wL1=np.absolute(diff)
        WL2=diff**2
        feature=hadamard
        if(G_test.has_edge(n1,n2)==True):
            test_1=np.vstack([test_0,hadamard])
        else:
            test_0=np.vstack([test_1,hadamard])
print i

0
0 pass
1 pass
2 3 4 5 6 7 8 9 10 pass
11 12 13 14 15 16 17 18 pass
19 20 21 22 pass
23 pass
24 25 26 pass
27 28 29 30 31 32 pass
33 34 pass
35 36 37 pass
38 39 40 41 42 43 pass
44 45 46 pass
47 pass
48 49 pass
50 51 pass
52 53 pass
54 pass
55 56 57 pass
58 59 60 61 62 pass
63 64 65 pass
66 67 68 69 pass
70 pass
71 pass
72 73 pass
74 75 76 77 78 79 80 81 82 83 84 85 86 87 pass
88 pass
89 90 91 92 93 94 pass
95 96 pass
97 98 pass
99 100 101 pass
102 103 104 pass
105 pass
106 107 pass
108 109 pass
110 111 pass
112 pass
113 pass
114 115 116 117 118 119 120 pass
121 122 pass
123 124 125 126 127 pass
128 pass
129 130 131 132 133 pass
134 pass
135 pass
136 137 pass
138 pass
139 pass
140 141 pass
142 143 144 145 146 pass
147 148 149 pass
150 pass
151 152 153 pass
154 155 156 157 pass
158 pass
159 160 pass
161 pass
162 pass
163 164 pass
165 166 pass
167 pass
168 pass
169 pass
170 pass
171 pass
172 pass
173 174 175 pass
176 177 pass
178 pass
179 180 181 182 183 pass
184 pass
185 186 187 pass
1

1330 1331 1332 1333 1334 1335 1336 1337 pass
1338 pass
1339 1340 1341 pass
1342 1343 pass
1344 pass
1345 pass
1346 1347 pass
1348 pass
1349 1350 pass
1351 pass
1352 pass
1353 pass
1354 pass
1355 pass
1356 1357 1358 1359 pass
1360 pass
1361 pass
1362 1363 1364 1365 1366 1367 pass
1368 1369 pass
1370 1371 pass
1372 1373 pass
1374 pass
1375 1376 1377 pass
1378 1379 1380 1381 pass
1382 1383 pass
1384 1385 1386 1387 pass
1388 1389 1390 1391 1392 1393 1394 1395 1396 pass
1397 1398 1399 1400 pass
1401 1402 1403 1404 1405 1406 pass
1407 1408 1409 pass
1410 pass
1411 1412 1413 1414 pass
1415 1416 pass
1417 1418 pass
1419 1420 pass
1421 1422 pass
1423 pass
1424 1425 1426 1427 pass
1428 1429 pass
1430 1431 1432 1433 1434 1435 pass
1436 1437 1438 1439 pass
1440 pass
1441 1442 1443 1444 1445 1446 1447 pass
1448 pass
1449 1450 pass
1451 1452 1453 pass
1454 pass
1455 pass
1456 pass
1457 1458 pass
1459 pass
1460 pass
1461 1462 pass
1463 1464 pass
1465 pass
1466 1467 1468 pass
1469 1470 1471 1472 1473 

2561 2562 2563 2564 2565 2566 2567 2568 2569 pass
2570 2571 pass
2572 2573 pass
2574 pass
2575 2576 2577 2578 pass
2579 pass
2580 2581 2582 2583 2584 2585 2586 2587 2588 2589 2590 pass
2591 2592 2593 2594 2595 pass
2596 pass
2597 2598 pass
2599 2600 pass
2601 2602 2603 pass
2604 pass
2605 2606 pass
2607 2608 2609 2610 pass
2611 2612 pass
2613 2614 2615 2616 2617 pass
2618 pass
2619 pass
2620 pass
2621 2622 2623 2624 pass
2625 2626 2627 pass
2628 2629 2630 2631 pass
2632 2633 2634 2635 2636 2637 2638 2639 pass
2640 2641 2642 pass
2643 2644 2645 2646 pass
2647 pass
2648 2649 2650 2651 2652 2653 2654 2655 2656 2657 pass
2658 pass
2659 2660 2661 2662 2663 pass
2664 2665 2666 2667 2668 2669 2670 2671 pass
2672 pass
2673 pass
2674 pass
2675 pass
2676 2677 2678 2679 2680 2681 2682 pass
2683 2684 2685 2686 2687 2688 pass
2689 2690 2691 2692 pass
2693 2694 pass
2695 pass
2696 pass
2697 pass
2698 2699 pass
2700 2701 2702 pass
2703 pass
2704 2705 pass
2706 2707 pass
2708 2709 2710 2711 pass
2712 

3728 3729 3730 3731 3732 pass
3733 3734 3735 3736 pass
3737 3738 pass
3739 3740 3741 pass
3742 3743 pass
3744 3745 3746 3747 3748 pass
3749 pass
3750 3751 3752 3753 3754 pass
3755 pass
3756 pass
3757 3758 3759 pass
3760 3761 pass
3762 pass
3763 pass
3764 3765 pass
3766 3767 pass
3768 3769 pass
3770 pass
3771 3772 3773 3774 3775 pass
3776 pass
3777 pass
3778 3779 3780 pass
3781 pass
3782 3783 pass
3784 3785 3786 3787 3788 3789 3790 pass
3791 3792 pass
3793 3794 3795 3796 pass
3797 3798 3799 pass
3800 pass
3801 pass
3802 pass
3803 pass
3804 3805 3806 3807 3808 3809 3810 3811 3812 3813 pass
3814 pass
3815 pass
3816 3817 pass
3818 pass
3819 3820 pass
3821 pass
3822 3823 3824 pass
3825 3826 3827 pass
3828 pass
3829 3830 pass
3831 pass
3832 pass
3833 pass
3834 3835 3836 3837 3838 pass
3839 pass
3840 3841 3842 3843 3844 pass
3845 3846 3847 3848 pass
3849 3850 3851 3852 3853 3854 3855 3856 3857 3858 3859 pass
3860 3861 3862 3863 3864 3865 pass
3866 pass
3867 3868 pass
3869 3870 3871 3872 pass


4888 4889 4890 4891 pass
4892 pass
4893 4894 pass
4895 4896 pass
4897 pass
4898 4899 pass
4900 4901 pass
4902 pass
4903 pass
4904 4905 pass
4906 4907 pass
4908 pass
4909 4910 4911 pass
4912 4913 4914 pass
4915 4916 4917 pass
4918 4919 pass
4920 4921 4922 pass
4923 pass
4924 pass
4925 pass
4926 4927 4928 4929 pass
4930 4931 pass
4932 4933 4934 4935 4936 4937 pass
4938 4939 4940 4941 4942 4943 4944 pass
4945 4946 pass
4947 4948 pass
4949 4950 pass
4951 4952 pass
4953 pass
4954 pass
4955 4956 pass
4957 4958 4959 4960 4961 4962 4963 pass
4964 4965 4966 4967 pass
4968 4969 4970 pass
4971 4972 4973 4974 pass
4975 4976 pass
4977 pass
4978 pass
4979 4980 4981 pass
4982 4983 pass
4984 pass
4985 pass
4986 4987 4988 pass
4989 pass
4990 4991 4992 4993 4994 4995 4996 4997 pass
4998 4999 5000 pass
5001 pass
5002 pass
5003 pass
5004 5005 pass
5006 5007 5008 pass
5009 5010 5011 pass
5012 5013 pass
5014 pass
5015 pass
5016 5017 pass
5018 5019 pass
5020 5021 5022 pass
5023 5024 pass
5025 pass
5026 pass


...

55677 55678 55679 pass
55680 55681 pass
55682 55683 pass
55684 pass
55685 55686 55687 pass
55688 55689 pass
55690 55691 55692 55693 pass
55694 55695 55696 55697 pass
55698 pass
55699 pass
55700 pass
55701 55702 pass
55703 pass
55704 pass
55705 pass
55706 55707 pass
55708 pass
55709 pass
55710 55711 55712 55713 55714 pass
55715 pass
55716 pass
55717 55718 pass
55719 pass
55720 55721 55722 55723 pass
55724 55725 pass
55726 pass
55727 55728 pass
55729 55730 55731 pass
55732 55733 pass
55734 pass
55735 55736 pass
55737 pass
55738 55739 pass
55740 55741 55742 pass
55743 55744 pass
55745 55746 55747 55748 pass
55749 pass
55750 55751 55752 pass
55753 55754 pass
55755 55756 55757 55758 pass
55759 pass
55760 55761 55762 pass
55763 pass
55764 55765 pass
55766 pass
55767 55768 pass
55769 pass
55770 55771 55772 pass
55773 pass
55774 pass
55775 55776 pass
55777 55778 55779 pass
55780 55781 55782 55783 pass
55784 pass
55785 55786 pass
55787 55788 pass
55789 pass
55790 55791 pass
55792 55793 55794 pa

56684 56685 56686 56687 56688 56689 pass
56690 56691 pass
56692 pass
56693 pass
56694 56695 56696 56697 56698 pass
56699 56700 pass
56701 56702 56703 56704 56705 pass
56706 56707 56708 56709 56710 56711 56712 56713 56714 pass
56715 56716 pass
56717 56718 56719 pass
56720 pass
56721 pass
56722 56723 pass
56724 56725 56726 pass
56727 56728 pass
56729 pass
56730 56731 56732 pass
56733 pass
56734 pass
56735 56736 56737 pass
56738 pass
56739 56740 56741 56742 56743 pass
56744 56745 pass
56746 pass
56747 56748 56749 56750 pass
56751 56752 56753 56754 pass
56755 pass
56756 56757 56758 56759 pass
56760 pass
56761 56762 56763 pass
56764 56765 56766 pass
56767 56768 56769 56770 pass
56771 pass
56772 pass
56773 56774 pass
56775 pass
56776 pass
56777 56778 pass
56779 pass
56780 56781 56782 pass
56783 56784 56785 pass
56786 56787 pass
56788 pass
56789 56790 56791 56792 56793 56794 pass
56795 pass
56796 56797 56798 pass
56799 56800 pass
56801 56802 56803 pass
56804 56805 56806 56807 pass
56808 56809

57741 57742 pass
57743 57744 57745 57746 pass
57747 pass
57748 57749 57750 57751 pass
57752 57753 pass
57754 pass
57755 57756 pass
57757 57758 pass
57759 pass
57760 pass
57761 57762 pass
57763 57764 pass
57765 pass
57766 57767 pass
57768 57769 57770 57771 57772 57773 57774 57775 57776 pass
57777 pass
57778 57779 pass
57780 pass
57781 57782 pass
57783 57784 57785 pass
57786 57787 pass
57788 pass
57789 57790 57791 pass
57792 57793 57794 pass
57795 pass
57796 pass
57797 57798 pass
57799 57800 57801 57802 pass
57803 57804 pass
57805 pass
57806 pass
57807 57808 57809 57810 57811 57812 pass
57813 pass
57814 pass
57815 57816 pass
57817 pass
57818 pass
57819 57820 57821 pass
57822 pass
57823 57824 57825 57826 57827 57828 57829 57830 pass
57831 57832 57833 57834 pass
57835 57836 57837 57838 pass
57839 pass
57840 57841 pass
57842 57843 pass
57844 57845 57846 57847 57848 pass
57849 pass
57850 pass
57851 pass
57852 57853 57854 57855 pass
57856 pass
57857 57858 57859 pass
57860 pass
57861 57862 pas

58788 58789 58790 58791 58792 pass
58793 58794 58795 58796 pass
58797 pass
58798 58799 58800 58801 58802 pass
58803 pass
58804 pass
58805 pass
58806 pass
58807 58808 58809 pass
58810 58811 58812 pass
58813 pass
58814 58815 pass
58816 pass
58817 58818 58819 58820 58821 58822 58823 58824 58825 pass
58826 58827 pass
58828 pass
58829 pass
58830 pass
58831 58832 pass
58833 pass
58834 58835 58836 58837 pass
58838 58839 58840 pass
58841 58842 58843 58844 58845 pass
58846 pass
58847 58848 58849 pass
58850 58851 pass
58852 pass
58853 pass
58854 58855 pass
58856 58857 pass
58858 pass
58859 58860 58861 58862 58863 pass
58864 58865 58866 58867 pass
58868 58869 58870 pass
58871 pass
58872 58873 58874 58875 pass
58876 pass
58877 58878 58879 58880 58881 pass
58882 pass
58883 pass
58884 58885 pass
58886 pass
58887 58888 pass
58889 pass
58890 pass
58891 pass
58892 pass
58893 58894 58895 pass
58896 58897 58898 58899 pass
58900 58901 pass
58902 58903 pass
58904 pass
58905 58906 pass
58907 pass
58908 pass

KeyboardInterrupt: 

In [13]:
train_0.shape

(0, 128)

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score
f1=f1_score(y_test, y_pred, average='macro')
precision=precision_score(y_test, y_pred, average='macro')
recall=recall_score(y_test, y_pred, average='macro')

In [ ]:
f1, precision, recall